In [1]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

load_dotenv()

openai.api_key  = os.getenv("OPENAI_API_KEY")

In [6]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("docs/payment.pdf")
pages = loader.load()

In [ ]:
len(pages)

In [ ]:
page = pages[0]
print(page.page_content[0:100])

In [5]:
page.metadata

{'source': './payment.pdf', 'page': 0}

In [8]:
## split based on text
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

In [62]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=20,
    chunk_overlap=0, 
    separators=["\n\n", "\n", " ", ""]
)

In [65]:
docs = text_splitter.split_documents(pages)


In [57]:
len(docs)

34

In [66]:
print(docs)

[Document(page_content='Trillium Montessori', metadata={'source': 'docs/payment.pdf', 'page': 0}), Document(page_content='- Leander', metadata={'source': 'docs/payment.pdf', 'page': 0}), Document(page_content='03/06/2024', metadata={'source': 'docs/payment.pdf', 'page': 0}), Document(page_content='This invoice is', metadata={'source': 'docs/payment.pdf', 'page': 0}), Document(page_content='paid', metadata={'source': 'docs/payment.pdf', 'page': 0}), Document(page_content='Invoice #75167153', metadata={'source': 'docs/payment.pdf', 'page': 0}), Document(page_content='Trillium Montessori', metadata={'source': 'docs/payment.pdf', 'page': 0}), Document(page_content='- Leander', metadata={'source': 'docs/payment.pdf', 'page': 0}), Document(page_content='1699 US183 North', metadata={'source': 'docs/payment.pdf', 'page': 0}), Document(page_content='Leander, TX, 78641', metadata={'source': 'docs/payment.pdf', 'page': 0}), Document(page_content='EIN#:', metadata={'source': 'docs/payment.pdf', 'p